In [286]:
import numpy as np 
import pandas as pd
import regex as re
import unicodedata as ud
import nltk
from nltk.stem.isri import ISRIStemmer
from nltk.tokenize import wordpunct_tokenize

## Movies Datasets

In [73]:
forties_df = pd.read_csv('1940s_data/1940s_movies.csv')
fifties_df = pd.read_csv('1950s_data/Movies_chronicled1950s.csv')
sixties_df = pd.read_csv('1960s_data/1960s_movies.csv')
seventies_df = pd.read_csv('1970s_data/1970s_movies.csv')

#### EDA functions

In [74]:
def describe_dataframe(df):
    
    for column in df.columns:
        print('Column Name: ', str(column))
        print(df[column].describe())
        print('------------------------------')

#### Cleansing functions

In [75]:
def change_empty_entries_to_NaN(df, column_name):
    
    for row_index in range(df.shape[0]):
        if '??' in str(df.loc[row_index, column_name]) or \
        df.loc[row_index, column_name] == 'nan':
            df.loc[row_index, column_name] = np.NaN
    return df
    

In [76]:
def change_empty_synopses_type(df, column_name):
    
    for row_index in range(df.shape[0]):
        cell_content = df.loc[row_index,column_name]
        if(type(cell_content)== float):
#             df.loc[row_index,column_name] = np.NaN
            print(df.loc[row_index,column_name])
#             print(type(df.loc[row_index,column_name]))
    return df

In [77]:
def unify_date_format(df):
    for row in range(df.shape[0]):
        
        release_day = df.iloc[row, 15]
        if len(str(release_day)) == 4:
            df.iloc[row,17] = release_day
            df.iloc[row,16] = 'يناير'
            df.iloc[row,15] = 1

    return df

In [78]:
def expand_release_date(df):
    # line continuation operators  '\' used for cleaner code
    df[['اليوم', 'الشهر', 'السنة']] = \
    df[' تاريخ العرض'].str.split(' ', expand = True)
    
    return df
    

In [79]:
def find_movies_with_missing_synopses(df):
    
    missing_movies = []
    for row_index in range(df.shape[0]):
        if(type(df.iloc[row_index, 3]) == float):
            missing_movies.append(df.iloc[row_index, 0])
            
    return missing_movies

In [80]:
def parse_line(line_to_parse):
    
            parsed_values = line_to_parse.split('||')
            movie_name_date = parsed_values[0].strip(')').split('(')
            
            name = movie_name_date[0]
            year = movie_name_date[1]
            df_feature = parsed_values[1]
            
            return name, year, df_feature

In [299]:
def add_scraped_features(df, file_path, column_name):
    
    with open(file_path, 'r') as filehandle:
        
        for line in filehandle:
            
            current_line = line[:-1]
            name, year, df_feature = parse_line(current_line)
            if(column_name == ' ملخص'):
               
                synopses_cell = df[column_name].loc[(df['اسم الفيلم'] == name) \
                 & (df['السنة'] == year)]
                
                try:
                    if(synopses_cell.values[0] == 'nan'):
                        df.loc[(df['اسم الفيلم'] == name) \
                     & (df['السنة'] == year), column_name] = df_feature
                except:
                    print(name)
            else:        

                df.loc[(df['اسم الفيلم'] == name) \
                     & (df['السنة'] == year), column_name] = df_feature
          
    return df

In [82]:
def filter_foregin_movies(df, file_path):
    
    with open(file_path, 'r') as filehandle:
            
        for line in filehandle:
            
            current_line = line[:-1]
            name, year, country = parse_line(current_line)
           
            if(not(('مصر') in country)):   
                movie_index = df.loc[(df['اسم الفيلم'] == name) \
                                     & (df['السنة'] == year)].index
                df.drop(inplace = True, index = movie_index)
    return df

In [83]:
def get_avg_short_movie_time(df):
    
    short_movies_durations = df[' مدة الفيلم (دقيقة)'].loc[ \
                                    df[' تصنيف الفيلم'] == 'ﻗﺼﻴﺮ']
    avg_duration = np.average(list(short_movies_durations))
    
    return avg_duration

## *_1940s Movies_*

### EDA

In [84]:
forties_df.sample(10)

,اسم الفيلم,تاريخ العرض,تصنيف الفيلم,مدة الفيلم (دقيقة),ملخص,تأليف,تمثيل,إنتاج,تصوير,مونتاج,ديكور,ملابس,موسيقى,إخراج,توزيع
50,نداء الدم,18 اكتوبر 1943,NaN,105.0,"شوقى رجل أعمال ثرى، تعمل لديه ""بثينة"" كسكرتيرة...","إبراهيم لاما,السيد زيادة","بدر لاما,روحية خالد,سلوى علام,منسى فهمي,إسماعي...",كوندروفيلم,NaN,NaN,NaN,NaN,NaN,إبراهيم لاما,"كوندروفيلم,ستوديو لاما"
321,السجينة رقم 17,1949,NaN,103.3,فتاة فقيرة تعيش مع أمها المريضة. تموت الأم فتس...,"عمر جميعى,أحمد جلال","ماري كويني,عماد حمدي,محمود المليجي,روحية خالد,...",ماري كويني,محمد عبدالعظيم,مارسيل صالح,NaN,NaN,يوسف صالح,عمر جميعى,منتخبات بهنا فيلم
204,أسير الظلام,9 يونيو 1947,NaN,108.0,تعيش فتاة يتيمة مع زوجة أبيها التي تعاملها بقس...,عز الدين ذو الفقار,"سراج منير,مديحة يسري,محمود المليجي,علوية جميل,...",NaN,NaN,NaN,NaN,NaN,NaN,عز الدين ذو الفقار,NaN
127,مجد ودموع,4 مارس 1946,ا????ا??,120.0,تدور الأحداث حول (درية) المطربة الشابة والتي ت...,"يوسف جوهر,أحمد بدرخان","نور الهدى,محمد فوزي,بشارة واكيم,زوزو ماضي,أمين...","اتحاد الفنيين,حلمي رفلة",عبدالحليم نصر,"ألبير نجيب,عبدالحليم نصر","ولي الدين سامح,سيد شاهين",NaN,"أحمد رامي,بيرم التونسي,صالح جودت,مأمون الشناوي...",أحمد بدرخان,منتخبات بهنا فيلم
199,شادية الوادي,7 ابريل 1947,NaN,125.0,وحيد فخرى فنان يسافر إلى باريس مع اثنين من ا...,يوسف وهبي,"ليلى مراد,يوسف وهبي,حسن فايق,أحمد علام,عبدالسل...",نحّاس فيلم,"سامى بريل,حسن داهش",ألبير نجيب,"حبيب خوري,بكر نصر",عبدالعزيز نصار,"رياض السنباطي,عبدالحليم نويرة,أحمد رامي,مصطفى ...","يوسف وهبي,كمال بركات",نحّاس فيلم
42,محطة الأنس,1942,NaN,108.0,تهرب (سحر) التي تهوى الغناء من بلدتها لكي تقاب...,"كمال سليم,موريس كساب,بديع خيري","عقيلة راتب,محمود المليجي,ماري منيب,علي الكسار,...",ستوديو مصر,"فيرى فاركاش (فرانسوا فاركاش),فرانسوا فاراكاش",NaN,NaN,NaN,NaN,عبدالفتاح حسن,ستوديو مصر
38,لو كنت غني,26 نوفمبر 1942,??????,100.0,تدور أحداث الفيلم حول حلاق فقير يدعى محروس (بش...,أبو السعود الإبياري,"بشارة واكيم,إحسان الجزايرلي,عبدالفتاح القصري,ي...",لوتس فيلم (آسيا وشركاها),عبدالحليم نصر,هنري بركات,عبدالحميد السخاوي,NaN,NaN,"هنري بركات,يوسف معلوف",منتخبات بهنا فيلم
241,صاحبة العمارة,18 مارس 1948,??????,115.0,فتاة فقيرة ترتبط بشاب فقير ويتفقان على الزواج ...,"عبدالفتاح حسن,بديع خيري","سامية جمال,محمد فوزي,إسماعيل يس,زينات صدقي,علي...",NaN,ف . فاراكاش,إحسان فرغل,عباس حلمي,NaN,محمد فوزي,عبدالفتاح حسن,شارل ليفشتز
176,سر أبي,1946,NaN,103.2,يسافر الصغير فؤاد ابن عادل باشا للخارج . يع...,"محمد مصطفى سامي,مصطفى حسن,بديع خيري","أنور وجدي,صباح,سراج منير,بشارة واكيم,محمود رضا...","مصطفى حسن,هليوبوليس فيلم",مصطفى حسن,كمال الشيخ,NaN,NaN,NaN,ولي الدين سامح,NaN
27,ابن الصحراء,11 فبراير 1942,NaN,110.0,يشتهر الشيخ معروف حاكم إحدى القبائل بالصحراء ب...,السيد زيادة,"بدر لاما,روحية خالد,إحسان الجزايرلي,مختار عثما...","أفلام لاما المصرية,ستوديو لاما",إبراهيم شيبا,"بدر عبدالله,أحمد إسماعيل",محروس زيادة,NaN,"يوسف صالح,سامى نعسان,سامي نعمان","إبراهيم لاما,السيد زيادة",نحّاس فيلم


In [85]:
describe_dataframe(forties_df)

Column Name:  اسم الفيلم
count             331
unique            330
top       خاتم سليمان
freq                2
Name: اسم الفيلم, dtype: object
------------------------------
Column Name:   تاريخ العرض
count      331
unique     216
top       1946
freq        24
Name:  تاريخ العرض, dtype: object
------------------------------
Column Name:   تصنيف الفيلم
count       127
unique        9
top       ??ا??
freq         46
Name:  تصنيف الفيلم, dtype: object
------------------------------
Column Name:   مدة الفيلم (دقيقة)
count    331.000000
mean     106.399698
std       11.097976
min       70.000000
25%      100.000000
50%      105.000000
75%      110.000000
max      152.000000
Name:  مدة الفيلم (دقيقة), dtype: float64
------------------------------
Column Name:   ملخص
count                                                   316
unique                                                  316
top       الحب اسمى معاني الحياة ..هذا ما حاولت الفتاة (...
freq                                           

In [86]:
forties_df.loc[forties_df['اسم الفيلم'] == 'خاتم سليمان']

,اسم الفيلم,تاريخ العرض,تصنيف الفيلم,مدة الفيلم (دقيقة),ملخص,تأليف,تمثيل,إنتاج,تصوير,مونتاج,ديكور,ملابس,موسيقى,إخراج,توزيع
174,خاتم سليمان,1946,NaN,103.2,NaN,أبو بثينة,أمينة شريف,أفلام حرب,NaN,NaN,NaN,NaN,NaN,فؤاد شبل,NaN
227,خاتم سليمان,1947,ا????ا??,117.0,تناول الفيلم في إطار من الخيال، قصة المعلم بيو...,"حسن رمزي,السيد زيادة","ليلى مراد,يحيى شاهين,زكي رستم,نيللي مظلوم,ثريا...",أفلام النصر (محمد حسن رمزي وشركاه),"مصطفى حسن,وحيد فريد","ألبير نجيب,وفيقة أبو جبل",حبيب خوري,قاسم وجدي,"حسين السيد,السيد زيادة,رياض السنباطي,محمد القص...","حسن رمزي,ألبير نجيب,عبدالرحمن شريف",NaN


In [87]:
forties_df[' تصنيف الفيلم'].value_counts()

??ا??       46
???????     26
??????      25
ا????ا??    16
?????        7
تاريخي       2
?????ا?      2
????         2
???          1
Name:  تصنيف الفيلم, dtype: int64

### Cleansing

In [88]:
forties_df = change_empty_entries_to_NaN(forties_df, ' تصنيف الفيلم')

In [89]:
forties_df[' ملخص'] = forties_df[' ملخص'].astype(str)

In [90]:
date_expanded_df_40s = expand_release_date(forties_df)
date_expanded_df_40s = unify_date_format(date_expanded_df_40s)

In [91]:
date_expanded_df_40s = date_expanded_df_40s.drop(columns=[' تاريخ العرض'])
date_expanded_df_40s['التقييم'] = np.nan

In [92]:
foreigns_filtered_df_40s = filter_foregin_movies(date_expanded_df_40s \
                                                , '1940s_data/countries_1940s.txt')

In [93]:
genre_added_40s_df = add_scraped_features(foreigns_filtered_df_40s \
                                          , '1940s_data/1940s_genres.txt' \
                                         , ' تصنيف الفيلم')

يوم سعيد(1940)||ﺭﻭﻣﺎﻧﺴﻲ
يوم سعيد
الباشمقاول(1940)||ﺩﺭاﻣﺎ
الباشمقاول
قلب المرأة(1940)||ﺭﻭﻣﺎﻧﺴﻲ
قلب المرأة
الورشة(1940)||ﺩﺭاﻣﺎ
الورشة
حياة الظلام(1940)||ﺩﺭاﻣﺎ
حياة الظلام
سي عمر(1941)||ﻛﻮﻣﻴﺪﻱ
سي عمر
ألف ليلة وليلة(1941)||ﺩﺭاﻣﺎ
ألف ليلة وليلة
انتصار الشباب(1941)||ﺩﺭاﻣﺎ
انتصار الشباب
امرأة خطرة(1941)||ﺩﺭاﻣﺎ
امرأة خطرة
الفرسان الثلاثة(1941)||ﻛﻮﻣﻴﺪﻱ
الفرسان الثلاثة
ليلى بنت مدارس(1941)||موسيقي / استعراضي
ليلى بنت مدارس
أحب الغلط(1942)||ﺩﺭاﻣﺎ
أحب الغلط
أولاد الفقراء(1942)||ﺩﺭاﻣﺎ
أولاد الفقراء
ليلة الفرح(1942)||ﺩﺭاﻣﺎ
ليلة الفرح
الستات في خطر(1942)||ﺩﺭاﻣﺎ
الستات في خطر
ممنوع الحب(1942)||ﻛﻮﻣﻴﺪﻱ
ممنوع الحب
رباب(1942)||ﺩﺭاﻣﺎ
رباب
ليلى(1942)||ﺩﺭاﻣﺎ
ليلى
على مسرح الحياة(1942)||ﺩﺭاﻣﺎ
على مسرح الحياة
المتهمة(1942)||ﺩﺭاﻣﺎ
المتهمة
بنت ذوات(1942)||ﺩﺭاﻣﺎ
بنت ذوات
علي بابا والأربعين حرامي(1942)||ﻛﻮﻣﻴﺪﻱ
علي بابا والأربعين حرامي
لو كنت غني(1942)||ﻛﻮﻣﻴﺪﻱ
لو كنت غني
بحبح في بغداد(1942)||ﻛﻮﻣﻴﺪﻱ
بحبح في بغداد
اخيرا تزوجت(1942)||ﺩﺭاﻣﺎ
اخيرا تزوجت
العامل(1943)||ﺩﺭاﻣﺎ
العامل
البؤساء(1943)||ﺩﺭاﻣﺎ
البؤساء
الطريق المس

In [94]:
genre_added_40s_df[' تصنيف الفيلم'].value_counts()

ﺩﺭاﻣﺎ                75
ﻛﻮﻣﻴﺪﻱ               23
موسيقي / استعراضي    13
ﺭﻭﻣﺎﻧﺴﻲ              12
ﺟﺮﻳﻤﺔ                 2
ﺗﺸﻮﻳﻖ ﻭﺇﺛﺎﺭﺓ          2
ﻋﺎﺋﻠﻲ                 2
ﻣﻐﺎﻣﺮاﺕ               1
ﺧﻴﺎﻝ                  1
ﺣﺮﺏ                   1
Name:  تصنيف الفيلم, dtype: int64

In [95]:
ratings_added_40s_df = add_scraped_features(genre_added_40s_df \
                                          , '1940s_data/1940s_ratings.txt' \
                                         , 'التقييم')

يوم سعيد(1940)||6.6
يوم سعيد
الباشمقاول(1940)||6
الباشمقاول
تحت السلاح(1940)||5.8
تحت السلاح
قلب المرأة(1940)||5.7
قلب المرأة
فتاة متمردة(1940)||5.1
فتاة متمردة
رجل بين امراتين(1940)||0
رجل بين امراتين
صرخة في الليل(1940)||5.3
صرخة في الليل
الورشة(1940)||5.1
الورشة
أصحاب العقول(1940)||5.8
أصحاب العقول
حياة الظلام(1940)||5
حياة الظلام
سي عمر(1941)||8.2
سي عمر
ليلى بنت الريف(1941)||6.4
ليلى بنت الريف
إلى الأبد(1941)||0
إلى الأبد
ألف ليلة وليلة(1941)||6.2
ألف ليلة وليلة
انتصار الشباب(1941)||6.6
انتصار الشباب
امرأة خطرة(1941)||0
امرأة خطرة
الفرسان الثلاثة(1941)||5.8
الفرسان الثلاثة
صلاح الدين الأيوبي(1941)||5.3
صلاح الدين الأيوبي
عاصفة على الريف(1941)||5.8
عاصفة على الريف
مصنع الزوجات(1941)||6.3
مصنع الزوجات
ليلى بنت مدارس(1941)||5.4
ليلى بنت مدارس
عريس من استنبول(1941)||5.7
عريس من استنبول
أحب الغلط(1942)||5.5
أحب الغلط
أولاد الفقراء(1942)||6.1
أولاد الفقراء
ليلة الفرح(1942)||5
ليلة الفرح
الستات في خطر(1942)||4.3
الستات في خطر
ممنوع الحب(1942)||5.4
ممنوع الحب
ابن الصحراء(1942)||0
ابن الصح

عدالة السماء(1948)||5
عدالة السماء
بلبل أفندي(1948)||5.8
بلبل أفندي
أحبك أنت(1949)||6.5
أحبك أنت
كلام الناس(1949)||5.8
كلام الناس
العيش والملح(1949)||6
العيش والملح
ولدي(1949)||5.4
ولدي
المجنونة(1949)||5.3
المجنونة
نادية(1949)||6.2
نادية
ذو الوجهين(1949)||4.3
ذو الوجهين
البيت الكبير(1949)||5.1
البيت الكبير
الناصح(1949)||5.5
الناصح
نص الليل(1949)||4.7
نص الليل
على أد لحافك(1949)||5.7
على أد لحافك
المصري أفندي(1949)||6.9
المصري أفندي
مبروك عليكي(1949)||5.4
مبروك عليكي
ست البيت(1949)||5.3
ست البيت
جواهر(1949)||4.7
جواهر
المرأة شيطان(1949)||5.7
المرأة شيطان
الليل لنا(1949)||5.7
الليل لنا
كرسي الاعتراف(1949)||7.5
كرسي الاعتراف
القاتلة(1949)||4.7
القاتلة
فاطمة وماريكا وراشيل(1949)||6.4
فاطمة وماريكا وراشيل
المرأة(1949)||4.2
المرأة
اجازة في جهنم(1949)||6.4
اجازة في جهنم
كل بيت له راجل(1949)||5.3
كل بيت له راجل
غزل البنات(1949)||8.5
غزل البنات
لهاليبو(1949)||5.6
لهاليبو
صاحبة الملاليم(1949)||5.9
صاحبة الملاليم
عقبال البكاري(1949)||5
عقبال البكاري
أوعى المحفظة(1949)||6.3
أوعى المحفظة
عفريته هان

In [96]:
synopses_added_40s_df = add_scraped_features(ratings_added_40s_df \
                                          , '1940s_data/1940s_synopses.txt' \
                                         , ' ملخص')

يوم سعيد(1940)||محمد كمال شاب بسيط ويعطف عليه أصحاب البيت الذي يسكن فيه. يلتقي في الترام بفتاة يحبها وتبادله مشاعره، إلا أن والدها يعترض على هذه العلاقة مستندًا إلى فقر هذا المطرب، وتحاول أمينة مع والدها الذي يوافق على مقابلته، لكن تصادف أن يوم المقابلة يوافق يوم احتفال عند الهانم التى يعمل عندها والد الفتاة. 
يوم سعيد
الباشمقاول(1940)||تدور أحداث الفيلم حول المحامي ممتاز الذي يهوى مغازلة النساء، رغم زواجه، ويستمر بخيانة زوجته، حتى تقرر أن تلقنه درسًا بالاتفاق مع صديقتها، حيث تتظاهر بخيانته مع رجل آخر، فهل تنطلي الحيلة على ممتاز ويصدق خيانة زوجته له؟ أم يفهم الدرس ويكف عن خيانتها؟
الباشمقاول
تحت السلاح(1940)||أمينة فتاة من أسرة عريقة، تقرر العائلة زواجها من ابن عمها حكيم حفاظًا على الثروة، وبما أن هذا الزواج قد تم فرضًا فكانا يعيشان كالأغراب، تمر السنوات وتقوم الحرب فيصاب عادل ابنها بالحمى فينقل للمستشفى، تزوره والدته، وتتقابل مع الضابط خالد الذي تبهره بجمالها وهو لا يعرف أنها زوجة صديقة حكيم.
تحت السلاح
قلب المرأة(1940)||تدور أحداث الفيلم حول (خيرية)، الفتاة الثرية التي كانت مخطوبة لأ

هل تعلم(1946)||
هل تعلم
173    nan
Name:  ملخص, dtype: object
خاتم سليمان(1946)||
خاتم سليمان
174    nan
Name:  ملخص, dtype: object
كوكب أميرة الصحراء(1946)||في أجواء صحراوية، تقع قصة حب بين الفاتنة كوكب، وشاب من قبيلة منافسة، ويحاول بعض الخصوم إفشال قصة الحب. 
كوكب أميرة الصحراء
كوكب أميرة الصحراء
synopses:  Series([], Name:  ملخص, dtype: object)
سر أبي(1946)||يسافر الصغير فؤاد ابن عادل باشا للخارج. يعتدي عادل باشا على أمينة أرملة عطية ناظر العزبة وعندما تخبره أنها حامل يطردها، تضع مولودتها نهاد وفي نفس اليوم تلد منيرة زوجة سكرتير الباشا وليدها محروس.
سر أبي
يد الله(1946)||يقع الشاب الصعيدي مبروك في حبائل زينات الفتاة العصرية المستهترة، ويستطيع إجبار الجميع على قبول زواجه منها، إلا أنها تمارس انحلالها بين أفراد الأسرة فتنجح في الإيقاع بالأخ الأصغر محروس في شباكها، ثم تهيم حبًا بمحروس وتخطط للتخلص من زوجها لتنال أخاه. تحاول الأم نفيسة أن تبعد شبح الدمار عن أسرتها.
يد الله
ملائكة في جهنم(1946)||يترك الأب ابنته التي يشك في بنوتها، فتقع في أيدي زعيم عصابة للسرقة والنشل، ونصبح الفتاة مثلهم

حلاوة(1949)||تكبر الابنة يتيمة الأم ولا تعرف أبيها الذي ترك أمها، وفي عملها الفني يبدأ عادل زميلها في الإعجاب بها. ولكن ممن يعجبون بها أيضاً والدها، الذي لا يعرف أحدهما الآخر.
حلاوة
325    nan
Name:  ملخص, dtype: object
منديل الحلو(1949)||سوء تفاهم يحدث بين الراقصة قطر الندى وعزيزة التي تعيش مع والدتها، وتحب الأسطى عزوز الذي يخطبها مما يثير حقد الأسطى نعيم الحلاق الذي يحبها أيضًا، يدبر مكيدة ويخبر عزوز أن عزيزة تعمل بملهى ليلي تحت اسم قطر الندى، فيهجرها عزوز، ويقع مليونير في حب قطر الندى ويهديها جوهرة ثمينة.
منديل الحلو
ارواح هائمة(1949)||أثناء حرب فلسطين، يسافر صحافي شاب كمراسل لجريدته بأنباء ميدان القتال لكن يحدث أن يتعرض إلى هجوم وتتقلب به السيارة في حريق، ويتشوه وجهه، وتنفر منه خطيبته، وينفر منه الجميع إلى أن تقع في حبه ممرضة في المستشفى.
ارواح هائمة
هدى(1949)||تعيش هدى الطالبة في بورسعيد مع والدها الأرمل فتحي المهداوي صاحب شركة للتوريد والتصدير، نعيمة الموظفة في الشركة تتحايل على فتحي حتى وقع في حبها وتزوجها، واعترضت هدى لفارق المستوى، عاملت زوجة الأب ابنة زوجها معاملة سيئة مما دع

In [215]:
synopses_added_40s_df = synopses_added_40s_df.reset_index(drop=True)

In [229]:
def check_synopsis_has_no_names(cast_list, synopsis):
    for actor in cast_list:
        if(actor in synopsis):
            synopsis = synopsis.replace(actor, '')
    return synopsis

In [280]:
def stem_synopsis(partial_df):
    
    stemmer = ISRIStemmer()
    for row_index in range(partial_df.shape[0]):
        
        synopsis = partial_df.loc[row_index, ' ملخص']
        cleaned_synopsis = ''
        for word in synopsis.split():
            cleaned_synopsis += stemmer.stem(word) + ' '
            
        partial_df.loc[row_index, ' ملخص'] = cleaned_synopsis 
        
    return partial_df

In [237]:
def preprocess_synopses(original_df, partial_df):
    for row_index in range(partial_df.shape[0]):
        
        cast = str(original_df.loc[row_index, ' تمثيل'])
        cast_list = list(cast.split('،'))
        synopsis = original_df.loc[row_index, ' ملخص']
        
        cleaned_synopsis = \
        check_synopsis_has_no_names(cast_list,synopsis )
        cleaned_synopsis = remove_arabic_characters(synopsis)
        
        partial_df.loc[row_index, ' ملخص'] = cleaned_synopsis
    
    return partial_df

In [232]:
def remove_arabic_characters(synopsis):
    cleaned_synopsis = \
    ''.join(char for char in synopsis if not ud.category(char).startswith('P'))
    
    return cleaned_synopsis

In [277]:
def remove_stop_words(partial_df):
    
    stop_words = set(nltk.corpus.stopwords.words("arabic"))
    stop_words.add('على')
    for row_index in range(partial_df.shape[0]):
        
        synopsis = partial_df.loc[row_index, ' ملخص']
        
        for word in synopsis.split():
            if word in stop_words:
                synopsis = synopsis.replace(word, '')
                
        partial_df.loc[row_index, ' ملخص'] = synopsis
    return partial_df


In [283]:
def remove_punctuation(partial_df):
    for row_index in range(partial_df.shape[0]):
        synopsis = partial_df.iloc[row_index, 2]
        if((synopsis) != 'nan'):
            synopsis = re.sub(r'\([^)]*\)', '', synopsis)
            synopsis = re.sub('".*?"', '', synopsis)
            synopsis = re.sub(r'[^\w\s]','',synopsis)

            partial_df.iloc[row_index, 2] = synopsis
    return partial_df

In [292]:
def tokenize_synopsis(partial_df):
    
    for row_index in range(partial_df.shape[0]):
        
        synopsis = partial_df.loc[row_index, ' ملخص']
        tokenized_synopsis = wordpunct_tokenize(synopsis)
        partial_df.loc[row_index, ' ملخص'] = tokenized_synopsis
    return partial_df
    

### Prepare Synopses for Doc2Vec

In [263]:
synopses_analysis_df_40s = \
synopses_added_40s_df[[ 'اسم الفيلم', ' تصنيف الفيلم', ' ملخص']]

In [296]:
synopses_analysis_df_40s = preprocess_synopses(synopses_added_40s_df \
                                              , synopses_analysis_df_40s)
synopses_analysis_df_40s = remove_punctuation(synopses_analysis_df_40s)
synopses_analysis_df_40s = remove_stop_words(synopses_analysis_df_40s)
synopses_analysis_df_40s = stem_synopsis(synopses_analysis_df_40s)

In [295]:
synopses_analysis_df_40s = tokenize_synopsis(synopses_analysis_df_40s)

In [297]:
synopses_analysis_df_40s.to_csv('1940s_data/dataset_40s.csv')

## *_1950s Movies_*

In [ ]:
fifties_df.sample(10)

In [ ]:
describe_dataframe(fifties_df)

In [ ]:
fifties_df[' تصنيف الفيلم'].value_counts()

In [ ]:
fifties_df[fifties_df.duplicated(subset = ['اسم الفيلم'])]

In [ ]:
fifties_df.loc[fifties_df['اسم الفيلم'] == 'غرام في إستانبول (غرام في إسطنبول)']

In [ ]:
fifties_df = change_empty_entries_to_NaN(fifties_df, ' تصنيف الفيلم')
fifties_df[' ملخص'] = fifties_df[' ملخص'].astype(str)

In [ ]:
date_expanded_df_50s = expand_release_date(fifties_df)
date_expanded_df_50s = unify_date_format(date_expanded_df_50s)

In [ ]:
date_expanded_df_50s = date_expanded_df_50s.drop(columns=[' تاريخ العرض'])
date_expanded_df_50s['التقييم'] = np.nan

In [ ]:
foreign_filtered_df_50s = filter_foregin_movies(date_expanded_df_50s
                                               , '1950s_data/countries_1950s.txt')

In [ ]:
ratings_added_df_50s = add_scraped_features(foreign_filtered_df_50s \
                                           , '1950s_data/1950s_ratings.txt'
                                           , 'التقييم')

In [ ]:
genres_added_df_50s = add_scraped_features(ratings_added_df_50s \
                                           , '1950s_data/1950s_genres.txt'
                                           , ' تصنيف الفيلم')

In [ ]:
synopses_added_50s_df = add_scraped_features(ratings_added_df_50s \
                                          , '1950s_data/1950s_synopses.txt' \
                                         , ' ملخص')

## *_1960s Movies_*

In [ ]:
sixties_df.sample(10)

In [ ]:
describe_dataframe(sixties_df)

In [ ]:
sixties_df.loc[sixties_df['اسم الفيلم'] == 'شايف خير']

In [ ]:
sixties_df[' تصنيف الفيلم'].value_counts()

In [ ]:
sixties_df.loc[sixties_df[' تصنيف الفيلم'] == 'ﻭﺛﺎﺋﻘﻲ']

In [ ]:
sixties_df = change_empty_entries_to_NaN(sixties_df, ' تصنيف الفيلم')
sixties_df[' ملخص'] = sixties_df[' ملخص'].astype(str)

_**Cleansing**_

In [ ]:
# sixties_df.drop_duplicates(keep = 'first', inplace = True)
sixties_df.drop(inplace = True, index = \
               sixties_df.loc[sixties_df[' تصنيف الفيلم'] == 'ﺳﻴﺮﺓ ﺫاﺗﻴﺔ']
                .index)

In [ ]:
sixties_df.loc[sixties_df['اسم الفيلم'] == 'الطريق']

In [ ]:
sixties_df.dtypes

In [ ]:
sixties_df.to_csv('1960s_data/1960s_no_duplicates.csv')

In [ ]:
date_expanded_60s_df = expand_release_date(sixties_df)
date_expanded_60s_df = unify_date_format(date_expanded_60s_df)

In [ ]:
date_expanded_60s_df = date_expanded_60s_df.drop(columns=[' تاريخ العرض'])

In [ ]:
date_expanded_60s_df['التقييم'] = np.nan

In [ ]:
date_expanded_1960s_df.to_csv('1960s_data/date_expanded_1960s.csv')

_**Add Webscraped Ratings**_

In [ ]:
foreigns_filtered_df_60s = filter_foregin_movies(date_expanded_60s_df \
                                                , '1960s_data/1960s_countries.txt')

In [ ]:
ratings_added_60s_df = add_scraped_features(date_expanded_60s_df \
                                    , '1960s_data/1960s_ratings.txt' \
                                    , 'التقييم')

In [ ]:
ratings_added_60s_df.to_csv('1960s_data/ratings_added_df_60s.csv')

In [ ]:
genre_added_60s_df = add_scraped_features(ratings_added_60s_df \
                                          , '1960s_data/1960s_genres.txt' \
                                         , ' تصنيف الفيلم')

In [ ]:
genre_added_60s_df.to_csv('1960s_data/genre_added_60s_df.csv')

In [ ]:
sixties_df.loc[sixties_df['اسم الفيلم'] == 'ورود ملونة']

In [ ]:
synopses_added_60s_df = add_scraped_features(genre_added_60s_df \
                                          , '1960s_data/1960s_synopses.txt' \
                                         , ' ملخص')

## *_1970s Movies_*

_**EDA**_

In [ ]:
describe_dataframe(seventies_df)

In [ ]:
seventies_df[' ملخص'].isna().sum()

In [13]:
seventies_df[' تصنيف الفيلم'].value_counts()

ﺩﺭاﻣﺎ           140
ﺭﻭﻣﺎﻧﺴﻲ          60
ﻛﻮﻣﻴﺪﻱ           56
ﺟﺮﻳﻤﺔ            22
اﺳﺘﻌﺮاﺿﻲ         15
تاريخي            8
ﺣﺮﻛﺔ              8
ﺗﺸﻮﻳﻖ ﻭﺇﺛﺎﺭﺓ      5
ﻗﺼﻴﺮ              5
ﻣﻐﺎﻣﺮاﺕ           4
ﺳﻴﺮﺓ ﺫاﺗﻴﺔ        4
ﻏﻤﻮﺽ              3
ﺣﺮﺏ               3
ﻭﺛﺎﺋﻘﻲ            3
ﺭﻳﺎﺿﻲ             2
ﺭﺳﻮﻡ ﻣﺘﺤﺮﻛﺔ       1
ﻋﺎﺋﻠﻲ             1
ﺧﻴﺎﻝ ﻋﻠﻤﻲ         1
Name:  تصنيف الفيلم, dtype: int64

_**Cleansing**_

In [14]:
seventies_df = change_empty_entries_to_NaN(seventies_df, ' تصنيف الفيلم')
seventies_df[' ملخص'] = seventies_df[' ملخص'].astype(str)

In [15]:
seventies_df[seventies_df.duplicated(subset = ['اسم الفيلم'])]

,اسم الفيلم,تاريخ العرض,تصنيف الفيلم,مدة الفيلم (دقيقة),ملخص,تأليف,تمثيل,إنتاج,تصوير,مونتاج,ديكور,ملابس,موسيقى,إخراج,توزيع
154,فرح زهران,1972,NaN,100.5,nan,محسن زايد,"عبدالعزيز مخيون,إنعام الجريتلي,ممدوح زايد",NaN,NaN,NaN,NaN,NaN,NaN,محسن زايد,NaN
164,فرح زهران,1972,NaN,100.5,nan,محسن زايد,"عبدالعزيز مخيون,إنعام الجريتلي,ممدوح زايد",NaN,NaN,NaN,NaN,NaN,NaN,محسن زايد,NaN
171,الظامئون,25 سبتمبر 1972,NaN,100.5,قرية عراقية تعانى العطش ويعد ابناؤها بالاتجاه ...,"خليل شوقي,ثامر مهدي,عبدالرزاق المطلبى","غازي الكناني,غازي التكريتي,خليل شوقي,ناهده الر...","العراق,المؤسسة المصرية العامة للسينما والتلفزي...",عبداللطيف صالح,"محمد شكري جميل,صبيح عبدالكريم",NaN,NaN,جميل بشير,محمد شكري جميل,NaN
172,فرح زهران,1972,NaN,100.5,nan,محسن زايد,"عبدالعزيز مخيون,إنعام الجريتلي,ممدوح زايد",NaN,NaN,NaN,NaN,NaN,NaN,محسن زايد,NaN
302,شلة المشاغبين,1973,ﻛﻮﻣﻴﺪﻱ,90,ريكو مخرج لفرقة مسرحية مصرية، تأتى رسالة من مت...,عصام المغربى,"صباح,عادل إمام,سعيد صالح,يونس شلبي,أحمد زكي,صل...",ستوديوهات هارون,علي خير الله,ميشال تامر,NaN,وليم خوري,NaN,"محمد سلمان,محمد مفتاح",مؤسسة صباح السينمائية
306,مخطوبتين بالوراثة,1974,NaN,108.3,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
365,مخطوبتين بالوراثة,1974,NaN,108.3,nan,NaN,محمد عوض,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
403,الحسناء و قاهر الفضاء,1975,NaN,95,nan,"محمد جومر,سهيل كنعان","محمود جبر,إغراء,أديب قدورة,نجاح حفيظ,أنور البا...","سميراميس فيلم,دمشق",محمد الرواس,مروان عكاوي,NaN,NaN,هنري,سهيل كنعان,NaN
414,الحسناء و قاهر الفضاء,1975,NaN,95,nan,"محمد جومر,سهيل كنعان","محمود جبر,إغراء,أديب قدورة,نجاح حفيظ,أنور البا...","سميراميس فيلم,دمشق",محمد الرواس,مروان عكاوي,NaN,NaN,هنري,سهيل كنعان,NaN
421,الحسناء و قاهر الفضاء,1975,NaN,95,nan,"محمد جومر,سهيل كنعان","محمود جبر,إغراء,أديب قدورة,نجاح حفيظ,أنور البا...","سميراميس فيلم,دمشق",محمد الرواس,مروان عكاوي,NaN,NaN,هنري,سهيل كنعان,NaN


In [16]:
seventies_df[seventies_df.duplicated(subset = ['اسم الفيلم', ' تمثيل'])]
seventies_df.drop_duplicates(subset = ['اسم الفيلم', ' تمثيل'], inplace= True, keep = 'last')

In [17]:
seventies_df.drop(inplace = True, index = 306)
seventies_df.drop(inplace = True, index = 156)
seventies_df.drop(inplace = True, index = 256)
seventies_df.drop(inplace = True, index = 389)
seventies_df.drop(inplace = True, index = 259)
seventies_df.drop(inplace = True, index = 433)
seventies_df.drop(inplace = True, index = 2)
seventies_df.drop(inplace = True \
                  , index = seventies_df.loc[ \
                                    seventies_df[' تصنيف الفيلم'] == 'ﺳﻴﺮﺓ ﺫاﺗﻴﺔ'].index)


In [18]:
seventies_df[seventies_df.duplicated(subset = ['اسم الفيلم'])]

,اسم الفيلم,تاريخ العرض,تصنيف الفيلم,مدة الفيلم (دقيقة),ملخص,تأليف,تمثيل,إنتاج,تصوير,مونتاج,ديكور,ملابس,موسيقى,إخراج,توزيع
757,الطيور المهاجرة,1979,ﺩﺭاﻣﺎ,90,يقرر علاء الهجرة إلى أستراليا، يفاتح زميلته، و...,بهيج إسماعيل,"شمس البارودي,حسن يوسف,سهير الباروني,نبيل بدر,ل...","أفلام حسن يوسف,يوسف عبيد","إبراهيم صالح,كمال كريم","فتحي داود,محمد ابراهيم,مارسيل صالح",NaN,NaN,عمر خورشيد,"حسن يوسف,عدلي يوسف,ملاك أندراوس","وكالة الجاعونى,أفلام حسن يوسف"
758,السراب,1979,NaN,-,nan,NaN,"عبدالله العمراني,محمد الرزين,محمد عفيفي,فاطمة ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
seventies_df.loc[seventies_df[' تصنيف الفيلم'] == 'ﺧﻴﺎﻝ ﻋﻠﻤﻲ']

,اسم الفيلم,تاريخ العرض,تصنيف الفيلم,مدة الفيلم (دقيقة),ملخص,تأليف,تمثيل,إنتاج,تصوير,مونتاج,ديكور,ملابس,موسيقى,إخراج,توزيع
166,العالم سنة 2000,1972,ﺧﻴﺎﻝ ﻋﻠﻤﻲ,105,يشعر الشاب بأنه غير قادر على الزواج من حبيبته ...,NaN,"حسن يوسف,سهير رمزي,زياد مولوي,أديب قدورة,نجوى ...","محمد الرواس,دمشق",محمد الرواس,مروان عكاوي,NaN,NaN,منوعات,أحمد فؤاد,NaN


In [38]:
seventies_df.loc[seventies_df['اسم الفيلم'] == 'سيدتي الجميلة']

,اسم الفيلم,تاريخ العرض,تصنيف الفيلم,مدة الفيلم (دقيقة),ملخص,تأليف,تمثيل,إنتاج,تصوير,مونتاج,ديكور,ملابس,موسيقى,إخراج,توزيع,اليوم,الشهر,السنة


In [39]:
date_expanded_70s_df = expand_release_date(seventies_df)
date_expanded_70s_df = unify_date_format(date_expanded_70s_df)

In [40]:
date_expanded_70s_df = date_expanded_70s_df.drop(columns=[' تاريخ العرض'])
date_expanded_70s_df['التقييم'] = np.nan

In [ ]:
date_expanded_70s_df.to_csv('1970s_data/date_expanded_1970s.csv')

In [41]:
foreign_filtered_df_70s = \
    filter_foregin_movies(date_expanded_70s_df, '1970s_data/1970s_countries.txt')

In [ ]:
foreign_filtered_df_70s.to_csv('1970s_data/foreign_movies_filtered.csv')

In [42]:
ratings_added_1970s_df = add_scraped_features(foreign_filtered_df_70s \
                                    , '1970s_data/1970s_ratings.txt' \
                                    , 'التقييم')

حكاية بسيطة كهذه(1970)||0
حكاية بسيطة كهذه
هورو كي كم أليس(1970)||0
هورو كي كم أليس
ابن أفريقيا(1970)||4.4
ابن أفريقيا
الغشاش(1970)||5.4
الغشاش
سر اللؤلؤة(1970)||5.8
سر اللؤلؤة
فرقة المرح(1970)||5.9
فرقة المرح
الساعات الرهيبة(1970)||5.3
الساعات الرهيبة
لا لا يا حبيبي(1970)||4.4
لا لا يا حبيبي
عين الحياة(1970)||6.6
عين الحياة
لسنا ملائكه(1970)||4.6
لسنا ملائكه
من عظماء الإسلام(1970)||5.5
من عظماء الإسلام
حكاية بسيطة كهذه(1970)||0
حكاية بسيطة كهذه
هورو كي كم أليس(1970)||0
هورو كي كم أليس
ابن أفريقيا(1970)||4.4
ابن أفريقيا
الغشاش(1970)||5.4
الغشاش
سر اللؤلؤة(1970)||5.8
سر اللؤلؤة
فرقة المرح(1970)||5.9
فرقة المرح
الساعات الرهيبة(1970)||5.3
الساعات الرهيبة
لا لا يا حبيبي(1970)||4.4
لا لا يا حبيبي
عين الحياة(1970)||6.6
عين الحياة
لسنا ملائكه(1970)||4.6
لسنا ملائكه
من عظماء الإسلام(1970)||5.5
من عظماء الإسلام
صراع مع الموت(1970)||4.8
صراع مع الموت
ربع دستة اشرار(1970)||5.9
ربع دستة اشرار
مكتوب(1970)||0
مكتوب
اللص الظريف(1970)||6.7
اللص الظريف
لصوص على موعد(1970)||0
لصوص على موعد
الصديقان(1970

علتنا فينا(1973)||0
علتنا فينا
شقة للحب(1973)||4.7
شقة للحب
بوبي(1973)||6.6
بوبي
مسك وعنبر(1973)||5
مسك وعنبر
صوت الحب(1973)||6
صوت الحب
الشمس لن تغرب عن مدينتي(1973)||0
الشمس لن تغرب عن مدينتي
بقايا صور(1973)||5.5
بقايا صور
البنات لازم تتجوز(1973)||6.2
البنات لازم تتجوز
الرجل الآخر(1973)||7
الرجل الآخر
رجال لا يخافون الموت(1973)||5.5
رجال لا يخافون الموت
مدرسة المراهقين(1973)||3.9
مدرسة المراهقين
زوجتي من الهيبز(1973)||5.4
زوجتي من الهيبز
الشحات(1973)||6.7
الشحات
شيء من الحب(1973)||5.6
شيء من الحب
ذات الوجهين(1973)||5.9
ذات الوجهين
ديسمبر(1973)||0
ديسمبر
عمال عبيد أرقاء: هؤلاء العرب جيرانكم(1973)||0
عمال عبيد أرقاء: هؤلاء العرب جيرانكم
أبناء للبيع(1973)||5.3
أبناء للبيع
امرأة من القاهرة(1973)||5.3
امرأة من القاهرة
سرب الابطال(1973)||5.5
سرب الابطال
جيتار الحب(1973)||5.5
جيتار الحب
مدرسة المشاغبين(1973)||5.4
مدرسة المشاغبين
شروال وميني جوب(1973)||5.2
شروال وميني جوب
الحب الذي كان(1973)||6.3
الحب الذي كان
نساء الليل(1973)||6.2
نساء الليل
زمان يا حب(1973)||5.4
زمان يا حب
طاحونة السيد فاب

مجانين بالوراثة(1975)||5.5
مجانين بالوراثة
الكل عاوز يحب(1975)||5.5
الكل عاوز يحب
الأستاذ أيوب(1975)||3.8
الأستاذ أيوب
شمس الضباع(1976)||0
شمس الضباع
الغدير(1976)||0
الغدير
عندنا كل الموت كى ننام(1976)||0
عندنا كل الموت كى ننام
الكروان له شفايف(1976)||4.6
الكروان له شفايف
الطيور المهاجرة(1976)||0
الطيور المهاجرة
بيت بلا حنان(1976)||6.1
بيت بلا حنان
الشرقي أو الصمت العنيف(1976)||0
الشرقي أو الصمت العنيف
حافية على جسر الذهب(1976)||8.4
حافية على جسر الذهب
رماد الزريبة(1976)||5.4
رماد الزريبة
سنه أولى حب(1976)||6.8
سنه أولى حب
رجل زائد عن الحاجة(1972)||0
رجل زائد عن الحاجة
صانع النجوم(1976)||5.5
صانع النجوم
الحياة نغم(1976)||5.3
الحياة نغم
الأحمر والأبيض والأسود(1976)||0
الأحمر والأبيض والأسود
سنوات العمر(1976)||5.7
سنوات العمر
صلب المسيح من قبل(1976)||0
صلب المسيح من قبل
المزيكا في خطر(1976)||5.6
المزيكا في خطر
الليل الطويل(1976)||5.8
الليل الطويل
وعادت الحياة(1976)||5.4
وعادت الحياة
ليل الرجال(1976)||5.9
ليل الرجال
لقاء هناك(1976)||6.3
لقاء هناك
شوق(1976)||5.4
شوق
جواز على الهوا(1976)||5

رجب فوق صفيح ساخن(1978)||7.6
رجب فوق صفيح ساخن
حساب السنين(1978)||5
حساب السنين
عيب يا لولو .. يا لولو عيب(1978)||6.5
عيب يا لولو .. يا لولو عيب
شيطان الجزيرة(1978)||5.3
شيطان الجزيرة
القمة الباردة(1978)||5.1
القمة الباردة
أولاد الحلال(1978)||4.5
أولاد الحلال
نجوم على الرصيف(1979)||0
نجوم على الرصيف
حياتي عذاب(1979)||6.4
حياتي عذاب
يوم آخر(1979)||5.3
يوم آخر
ولا عزاء للسيدات(1979)||6.9
ولا عزاء للسيدات
خطيئة ملاك(1979)||5.4
خطيئة ملاك
المصيدة(1979)||5.3
المصيدة
يوم في حياة عباس(1979)||0
يوم في حياة عباس
حبيبتي ياحب التوت(1979)||4.6
حبيبتي ياحب التوت
صراع(1979)||0
صراع
القلعة الخامسة(1979)||0
القلعة الخامسة
البنت كبرت(1979)||4.7
البنت كبرت
حكاية قرية وحرب(1979)||5.7
حكاية قرية وحرب
عزيزة(1979)||0
عزيزة
التقدم إلى العمق(1979)||0
التقدم إلى العمق
المتوحشة(1979)||5.6
المتوحشة
لا تبكي يا حبيب العمر(1979)||6.9
لا تبكي يا حبيب العمر
قناة السويس(1979)||0
قناة السويس
قصة الحي الغربي(1979)||4.4
قصة الحي الغربي
خلي بالك من جيرانك(1979)||7.5
خلي بالك من جيرانك
السيد ناتوارالال(1979)||0
السيد ناتوا

In [ ]:
ratings_added_1970s_df.to_csv('1970s_data/ratings_added_70s_df.csv')

In [44]:
genre_added_df_70s = add_scraped_features(ratings_added_1970s_df \
                                          , '1970s_data/1970s_genres.txt'
                                         ,' تصنيف الفيلم')

ابن أفريقيا(1970)||ﺗﺸﻮﻳﻖ ﻭﺇﺛﺎﺭﺓ
ابن أفريقيا
الغشاش(1970)||ﺩﺭاﻣﺎ
الغشاش
فرقة المرح(1970)||موسيقي / استعراضي
فرقة المرح
عين الحياة(1970)||ﺟﺮﻳﻤﺔ
عين الحياة
لسنا ملائكه(1970)||ﺩﺭاﻣﺎ
لسنا ملائكه
من عظماء الإسلام(1970)||تاريخي
من عظماء الإسلام
صراع مع الموت(1970)||ﻛﻮﻣﻴﺪﻱ
صراع مع الموت
اللص الظريف(1970)||ﺩﺭاﻣﺎ
اللص الظريف
الحب والثمن(1970)||ﺗﺸﻮﻳﻖ ﻭﺇﺛﺎﺭﺓ
الحب والثمن
هروب(1970)||ﺩﺭاﻣﺎ
هروب
الأشرار(1970)||ﺗﺸﻮﻳﻖ ﻭﺇﺛﺎﺭﺓ
الأشرار
نهاية الشياطين(1970)||ﺣﺮﻛﺔ
نهاية الشياطين
الثعلب والحرباء(1970)||ﻏﻤﻮﺽ
الثعلب والحرباء
المرايه(1970)||ﻛﻮﻣﻴﺪﻱ
المرايه
الرجل المناسب(1970)||ﺩﺭاﻣﺎ
الرجل المناسب
رجال بلا ملامح(1970)||ﺩﺭاﻣﺎ
رجال بلا ملامح
سارق المحفظة(1970)||ﺩﺭاﻣﺎ
سارق المحفظة
أنا وزوجتي والسكرتيرة(1970)||ﻛﻮﻣﻴﺪﻱ
أنا وزوجتي والسكرتيرة
مغامرة شباب(1970)||ﻛﻮﻣﻴﺪﻱ
مغامرة شباب
نحن لا نزرع الشوك(1970)||ﺩﺭاﻣﺎ
نحن لا نزرع الشوك
الحب الضائع(1970)||ﺩﺭاﻣﺎ
الحب الضائع
شقة مفروشة(1970)||ﺗﺸﻮﻳﻖ ﻭﺇﺛﺎﺭﺓ
شقة مفروشة
سفاح النساء(1970)||ﻛﻮﻣﻴﺪﻱ
سفاح النساء
هاربات من الحب(1970)||ﺩﺭاﻣﺎ
هاربات من الحب
أنت إللي قتلت بابايا(1970)||ﺟﺮﻳﻤﺔ
أن

جواز على الهوا(1976)||ﻛﻮﻣﻴﺪﻱ
جواز على الهوا
ليتني ما عرفت الحب(1976)||ﺩﺭاﻣﺎ
ليتني ما عرفت الحب
المنحرفون(1976)||ﺩﺭاﻣﺎ
المنحرفون
ملك التاكس(1976)||ﻛﻮﻣﻴﺪﻱ
ملك التاكس
عودة الابن الضال(1976)||ﺩﺭاﻣﺎ
عودة الابن الضال
الراعية الحسناء(1976)||ﺭﻭﻣﺎﻧﺴﻲ
الراعية الحسناء
نساء تحت الطبع(1976)||ﺩﺭاﻣﺎ
نساء تحت الطبع
عمر قتلاتو(1976)||ﺩﺭاﻣﺎ
عمر قتلاتو
أموت مرتين وأحبك(1976)||ﺩﺭاﻣﺎ
أموت مرتين وأحبك
أزواج طائشون(1976)||ﻛﻮﻣﻴﺪﻱ
أزواج طائشون
الصمت(1976)||ﺩﺭاﻣﺎ
الصمت
فيفا زلاطا(1976)||ﻛﻮﻣﻴﺪﻱ
فيفا زلاطا
وبالوالدين إحسانا(1976)||ﺩﺭاﻣﺎ
وبالوالدين إحسانا
رحلة الأيام(1976)||ﺩﺭاﻣﺎ
رحلة الأيام
حكمتك يارب(1976)||ﺩﺭاﻣﺎ
حكمتك يارب
أخواته البنات(1976)||ﺩﺭاﻣﺎ
أخواته البنات
عالم عيال عيال(1976)||ﻛﻮﻣﻴﺪﻱ
عالم عيال عيال
سيقان في الوحل(1976)||ﺩﺭاﻣﺎ
سيقان في الوحل
دقة قلب(1976)||ﺩﺭاﻣﺎ
دقة قلب
أمواج بلا شاطئ(1976)||ﺩﺭاﻣﺎ
أمواج بلا شاطئ
وداعا إلى الأبد(1976)||ﺭﻭﻣﺎﻧﺴﻲ
وداعا إلى الأبد
الرسالة(1976)||ﻣﻐﺎﻣﺮاﺕ
الرسالة
قمر الزمان(1976)||ﻋﺎﺋﻠﻲ
قمر الزمان
دائرة الانتقام(1976)||ﻣﻐﺎﻣﺮاﺕ
دائرة الانتقام
أنا لا عاقلة ولا مجنونة(1976)||ﺩﺭاﻣﺎ


In [69]:
synopses_added_70s_df = add_scraped_features(genre_added_df_70s \
                                          , '1970s_data/1970s_synopses.txt' \
                                         , ' ملخص')

## So far, 1970s movies have their genres, ratings, and are filtered from foreign movies.

In [ ]:
foreigns_filtered_df_70s.to_csv('1970s_data/foreigns_filtered_df_70s.csv')

In [ ]:
date_expanded_70s_df

In [ ]:
date_expanded_70s_df.shape[0]